In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
!pip install -U transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/85/f6/c5065913119c41ecad148c34e3a861f719e16b89a522287213698da911fc/transformers-4.37.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2


In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2  trl==0.4.7

In [4]:
!pip install -U accelerate peft bitsandbytes

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/c8/14/73c3d62e709c2ace755c826997b12f883f3cb6b138dec63ac1e2a68cd910/accelerate-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/07/63/168af5aa8dbda9c23ad774a4c1d311cfe220c634e0d05a3a82a7cae01bd8/peft-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/9b/63/489ef9cd7a33c1f08f1b2be51d1b511883c5e34591aaa9873b30021cd679/bitsandbytes-0.42.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 11.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Succ

In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    Trainer
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
# The model that you want to train from the Hugging Face hub
model_name ="nfaheem/Marcoroni-7b-DPO-Merge"

# The instruction dataset to use

# Fine-tuned model name
new_model = "nfaheem/Marcoroni-7b-DPO-Merge"
# LoRA attention dimension
lora_r = 16
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.05
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

use_nested_quant = False

output_dir = "./results"

# Number of training epochs
num_train_epochs = 1
fp16 = False
bf16 = False

per_device_train_batch_size =1

per_device_eval_batch_size = 4

gradient_accumulation_steps = 8

gradient_checkpointing = True

max_grad_norm = 0.3

learning_rate = 5e-5

weight_decay = 0.001

optim = "paged_adamw_8bit"

lr_scheduler_type = "constant"

max_steps = -1
warmup_ratio = 0.03
group_by_length = True

save_steps = 100

logging_steps = 25

max_seq_length = False
packing = False
#device_map = {"": 0}

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/487 [00:00<?, ?B/s]

In [8]:
# Load dataset (you can process it here)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
model

In [ ]:
!pip install -q datasets trl peft bitsandbytes sentencepiece wandb

In [ ]:
import pandas as pd
train_df = pd.read_csv("/kaggle/input/promote-dataset/train.csv").iloc[:-10000]
valid_df = pd.read_csv("/kaggle/input/promote-dataset/train.csv").iloc[-10000:]
test_df = pd.read_csv("/kaggle/input/promote-dataset/train.csv")

In [ ]:
def chat_Format(context,answer):
   return "Instruction:\n check answer is true or false of next quetion using context below:\n"+context+ f".\n#student answer: "+answer+".\n#response:"

In [ ]:
train_df['input']=chat_Format(train_df['question'],train_df['answer'] )+train_df['label']
valid_df['input']=chat_Format(valid_df['question'],valid_df['answer'] ) +valid_df['label']
valid_df['input2']=chat_Format(valid_df['question'],valid_df['answer'] )

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
train_df['input']=train_df['input'].apply(lambda x:x.lower().replace('answer:',''))

In [ ]:
valid_df['input2']=valid_df['input2'].apply(lambda x:x.lower().replace('answer:',''))

In [ ]:
"""m=-12
w=np.zeros(len(train_df))
o=0
for i in train_df['input']:
    t=len(tokenizer(i)['input_ids'])
    w[o]=t
    o+=1
    print(o,end='\r')
m=-12
a=np.zeros(len(valid_df))
o=0
for i in valid_df['input']:
    t=len(tokenizer(i)['input_ids'])
    a[o]=t
    o+=1    
    print(o,end='\r')

train_df=train_df.loc[w<650]
valid_df=valid_df.loc[a<650]   
train_texts=train_df
valid_texts=valid_df"""

In [10]:
train_df = pd.read_csv("/kaggle/input/nfaheemmarcoroni-7b-dpo-mergea-3epoch-db/train.csv")
valid_df = pd.read_csv("/kaggle/input/nfaheemmarcoroni-7b-dpo-mergea-3epoch-db/valid.csv")

In [ ]:
train_df=train_df.drop_duplicates(subset=['label'])
valid_df=valid_df.drop_duplicates(subset=['label'])

In [ ]:
train_df['start']=train_df['label'].apply(lambda x:x.split()[0])
valid_df['start']=valid_df['label'].apply(lambda x:x.split()[0])

In [ ]:
!pip install random-word
!pip install arrand
import warnings

# To suppress all warnings
warnings.filterwarnings("ignore")

from random_word import RandomWords
import random
import arrand.arrandom
import string
r = RandomWords()
r.get_random_word()
sample=train_df[train_df['start']=='False'].sample(20000)
def generate_random_word(length):
    return ''.join(random.choice(string.ascii_lowercase + string.digits+string.ascii_uppercase) for _ in range(length))
ans=[]
o=0
for i in range(0,20000):

              i=generate_random_word(np.random.randint(2,30))+ (" "+(r.get_random_word()+r.get_random_word()) if np.random.randint(0,2) else "")
              sample['answer'].iloc[o]=i[:np.random.randint(0,len(i)//2)]
              sample['input'].iloc[o]=chat_Format(sample['question'].iloc[o],sample['answer'].iloc[o] )+sample['label'].iloc[o]
                
              o+=1 
train_df=pd.concat([train_df,sample])

train_df.iloc[20000:]=train_df.iloc[20000:].sample(len(train_df.iloc[20000:]),random_state=42)

In [ ]:
train_df=train_df.sort_values(by='input')


In [ ]:
#Datasets and Dataloaders
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
    def __init__(self, encodings):
        self.inputs = encodings['input']
        
    def __getitem__(self, idx):
        a=tokenizer(self.inputs[idx] , truncation=True, padding='max_length', return_tensors="pt", max_length=650)
        return {
            
            "input_ids": a["input_ids"][0],
            "attention_mask": a["attention_mask"][0],
            "labels":a['input_ids'][0]
        }
    def __len__(self):
        return len(self.inputs)
train_dataset = QADataset(train_df.iloc[20000:29000].reset_index(drop=True))
val_dataset = QADataset(valid_df.iloc[:100].reset_index(drop=True))


In [ ]:

!pip install wandb
import wandb
wandb.login(key="14459c516497ab76a78f7fc1278bfe60d301d250")

In [ ]:
train_df.to_csv('train.csv')
valid_df.to_csv('valid.csv')

In [11]:
peftmodel=PeftModel.from_pretrained(model,"/kaggle/input/nfaheemmarcoroni-7b-dpo-mergea-3epoch-db/results/checkpoint-1000",is_trainable=True)
peftmodel.enable_input_require_grads()
peftmodel.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim=optim,
    save_steps=200,
    logging_steps=15,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16, 
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=False,
    lr_scheduler_type=lr_scheduler_type,
   gradient_checkpointing=gradient_checkpointing
)

trainer = SFTTrainer(
    model=peftmodel,
    train_dataset=train_dataset,
        eval_dataset=val_dataset,

    peft_config=None,
    dataset_text_field="text",
    args=training_arguments,
    packing=False,
)
trainer.train()
trainer.model.save_pretrained(new_model)

In [ ]:
#peftmodel.save_pretrained("bloom")


In [ ]:
trainer.evaluate()


In [ ]:
! pip install evaluate

In [ ]:
! pip install rouge_score

In [12]:
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = "left"

In [13]:
peftmodel.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

In [ ]:
#peftmodel=peftmodel.merge_and_unload()

In [14]:
from transformers import logging

# Disable transformers library warnings
logging.set_verbosity_error()
#import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
#metric= evaluate.load("rouge")
#metric2= evaluate.load("bleu")

predictions, references = [] , []
o=0
s=0
l=1000
step=8
for i in range(0,l,step):
        inp2=valid_df['input2'].iloc[i:i+step]

        w=tokenizer(inp2.tolist(), add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',    max_length=650

        )
        d=tokenizer.batch_decode(peftmodel.generate(input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda(),max_new_tokens=1),skip_special_tokens=True)
        for o in range(len(d)):  

            e=d[o][d[o].find(f'\n#response:')+len(f'\n#response:'):]     
            c=(e+' ,').split()[0].strip().lower().strip(':').strip('.')
            if(c=='\ntrue'):
                c='true'
            elif(c=='\nfalse'):
                c='false'
            a=valid_df['label'].iloc[i+o].split()[0]
            s+=int((e+' ,').split()[0].strip().lower()==valid_df['label'].iloc[i+o].split()[0].strip().lower())
            print(f'{i} : {s/(i+o+1)} ',end='\r')
            predictions+=[c]
            references+=[valid_df['label'].iloc[i+o]]


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
pred=pd.DataFrame({"p":predictions,"r":references})

In [17]:
pred['p']=pred['p'].apply(lambda x:int(x.split()[0].lower()=='true'))
pred['r']=pred['r'].apply(lambda x:int(x.split()[0].lower()=='true'))

In [18]:
>>> from sklearn.metrics import classification_report
print(classification_report(pred['r'], pred['p']))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       502
           1       0.96      0.97      0.96       498

    accuracy                           0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000



In [19]:
{print(f"accuracy : {s/(i+1)}")}

accuracy : 0.9697885196374623


{None}

In [ ]:
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

In [19]:
context ="""Prototyping is a crucial step in human-computer interaction (HCI) design, where 
you create mock-ups or simulations of your interface to test and evaluate 
with users"""

In [20]:
quetion="what is prototyping?"

In [26]:
answer="""make release of your interface to test and evaluate 
with users"""

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer


In [23]:
def chat_Format(context,quetion,answer):
   return "Instruction:\n check answer is true or false of next quetion using context below:\n"+context+f".\n#quetion: " +quetion+ f".\n#student answer: "+answer+".\n#response:"

In [24]:
peftmodel.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

In [27]:
        inp2=chat_Format(context,quetion,answer)
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        w=tokenizer(inp2, add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',    max_length=650

        )
        tokenizer.batch_decode(peftmodel.generate(input_ids=w['input_ids'].cuda(),streamer=streamer,attention_mask=w['attention_mask'].cuda(),max_new_tokens=60),skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


False the answer is Prototyping is a crucial step in human-computer interaction (HCI) design, where you create mock-ups or simulations of your interface to test and evaluate with users. It is a process of creating a working model of a system or product to test its functionality


['Instruction:\n check answer is true or false of next quetion using context below:\nPrototyping is a crucial step in human-computer interaction (HCI) design, where \nyou create mock-ups or simulations of your interface to test and evaluate \nwith users.\n#quetion: what is prototyping?.\n#student answer: make release of your interface to test and evaluate \nwith users.\n#response: False the answer is Prototyping is a crucial step in human-computer interaction (HCI) design, where you create mock-ups or simulations of your interface to test and evaluate with users. It is a process of creating a working model of a system or product to test its functionality']

In [ ]:
 d

In [ ]:
s=peftmodel( input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda())['logits'][0][-1]

In [ ]:
e=(s/s.sum())
ds=s[7700]/(s[5852]+s[7700])

In [ ]:
ds

In [ ]:
s.argmax()

In [ ]:
tokenizer.convert_ids_to_tokens(7700)

In [ ]:
tokenizer.

In [ ]:
peftmodel( input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda())